In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

label_cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

df_test = pd.read_csv("/content/drive/MyDrive/toxic_data/test.csv")
df_test_labels = pd.read_csv("/content/drive/MyDrive/toxic_data/test_labels.csv")

df_test = pd.merge(df_test, df_test_labels, on="id")
df_test = df_test[df_test[label_cols].ne(-1).all(axis=1)]
df_test["label"] = (df_test[label_cols].sum(axis=1) > 0).astype(int)

texts = df_test["comment_text"].astype(str).tolist()
labels = df_test["label"].values

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model.to("cuda")
model.eval()

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
import torch

encodings = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")
dataset = TensorDataset(
    encodings["input_ids"],
    encodings["attention_mask"],
    torch.tensor(labels)
)
loader = DataLoader(dataset, batch_size=32)

In [ ]:
preds = []

with torch.no_grad():
    for batch in loader:
        input_ids, attention_mask, _ = [b.to("cuda") for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds.extend(torch.argmax(logits, dim=1).cpu().numpy())

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

print("Accuracy:", accuracy_score(labels, preds))
print("F1 (toxic):", f1_score(labels, preds))  # binary = class 1
print(classification_report(labels, preds))

In [ ]:
from peft import AutoPeftModelForSequenceClassification
from transformers import AutoTokenizer
import torch

model = AutoPeftModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/models/lora-toxic-roberta")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/models/lora-toxic-roberta")

model.eval().to("cuda")

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

encodings = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")
dataset = TensorDataset(
    encodings["input_ids"],
    encodings["attention_mask"],
    torch.tensor(labels)
)
loader = DataLoader(dataset, batch_size=32)

In [ ]:
preds = []
model.eval()

with torch.no_grad():
    for batch in loader:
        input_ids, attention_mask, _ = [b.to("cuda") for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds.extend(torch.argmax(logits, dim=1).cpu().numpy())

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

print("Accuracy:", accuracy_score(labels, preds))
print("F1 Score:", f1_score(labels, preds))
print(classification_report(labels, preds))

In [ ]:
all_probs = []
all_labels = []

with torch.no_grad():
    for batch in loader:
        input_ids, attention_mask, label = [b.to("cuda") for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)[:, 1]
        all_probs.extend(probs.cpu().numpy())
        all_labels.extend(label.cpu().numpy())

In [ ]:
import numpy as np

print("F1 scores at different thresholds:")
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
for t in thresholds:
    preds = (np.array(all_probs) > t).astype(int)
    score = f1_score(all_labels, preds)
    print(f"Threshold: {t:.1f} → F1 Score: {score:.4f}")

In [ ]:
from sklearn.metrics import f1_score, classification_report, precision_recall_curve

prec, rec, thres = precision_recall_curve(all_labels, all_probs)
f1s = 2 * (prec * rec) / (prec + rec + 1e-8)
best_idx = f1s.argmax()
best_threshold = thres[best_idx]
best_f1 = f1s[best_idx]

print(f"Best Threshold: {best_threshold:.3f}")
print(f"Best F1 Score: {best_f1:.4f}")

final_preds = (np.array(all_probs) > best_threshold).astype(int)
print("\nFinal classification report at best threshold:")
print(classification_report(all_labels, final_preds))

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch
from sklearn.metrics import accuracy_score, f1_score, classification_report

model_name = "martin-ha/toxic-comment-model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.to("cuda")
model.eval()

encodings = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

dataset = TensorDataset(encodings["input_ids"], encodings["attention_mask"], torch.tensor(labels))
loader = DataLoader(dataset, batch_size=32)

preds = []
with torch.no_grad():
    for batch in loader:
        input_ids, attention_mask, _ = [b.to("cuda") for b in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds.extend(torch.argmax(logits, dim=1).cpu().numpy())

print("Accuracy:", accuracy_score(labels, preds))
print("F1 Score:", f1_score(labels, preds))
print(classification_report(labels, preds))